In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost 

from sklearn.metrics import mean_squared_error


from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.preprocessing import StandardScaler

from time import time
from google.colab import drive
from time import perf_counter
import pickle
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
path_df_2016="/gdrive/MyDrive/MLDM Project/data/X_all_2016_final.zip"
df_train = pd.read_csv(path_df_2016)

# path_df_2017="/gdrive/MyDrive/MLDM Project/data/X_all_2016_final.zip"
# df_val = pd.read_csv(path_df_2017)

In [4]:
x_train, y_train = df_train.drop("ground_truth", axis=1), df_train["ground_truth"]
x_train.drop(["day","Id","station_id","hour","next hour precipitation (kg/m^2)"], axis=1, inplace=True)
# x_train.drop(["day","Id"], axis=1, inplace=True)
# x_train = pd.DataFrame(StandardScaler().fit_transform(x_train), columns=x_train.columns)
del df_train

# x_val, y_val = df_val.drop("ground_truth", axis=1), df_val["ground_truth"]
# x_val.drop(["day","station_id","hour","next hour precipitation (kg/m^2)"], axis=1, inplace=True)
# # x_train.drop(["day","Id"], axis=1, inplace=True)
# # x_train = pd.DataFrame(StandardScaler().fit_transform(x_train), columns=x_train.columns)
# del df_val

In [5]:


# "novembre décembre janvier février mars avril"
# +
# "mai juin juillet août septembre octobre"
def data_cluster(X,y):
  # cluster_1_index = X.loc[(df['A'] > 10) & (df['B'] < 15)]
  cluster_1_index = X[(X["month"]==1)].index.values
  np.append(cluster_1_index,X[(X["month"]==2)].index)
  cluster_1_index = np.append(cluster_1_index,X[(X["month"]==3)].index)
  cluster_1_index= np.append(cluster_1_index,X[(X["month"]==4)].index)
  cluster_1_index= np.append(cluster_1_index,X[(X["month"]==11)].index)
  cluster_1_index= np.append(cluster_1_index,X[(X["month"]==12)].index)


  cluster_2_index = X[(X["month"]==5)].index.values
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==6)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==7)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==8)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==9)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==10)].index)

  # print(X['month'])
  X_cluster1 = X.loc[cluster_1_index]
  y_cluster1 = y.loc[cluster_1_index]

  X_cluster2 = X.loc[cluster_2_index]
  y_cluster2 = y.loc[cluster_2_index]

  return X_cluster1, y_cluster1, X_cluster2, y_cluster2

X_cluster1, y_cluster1, X_cluster2, y_cluster2 = data_cluster(x_train,y_train)
month1 = X_cluster1['month']
month2 = X_cluster2['month']

X_cluster1.drop(["month"], axis=1, inplace=True)
X_cluster2.drop(["month"], axis=1, inplace=True)

In [ ]:


X_cluster1 = pd.DataFrame(StandardScaler().fit_transform(X_cluster1), columns=X_cluster1.columns)
X_cluster2 = pd.DataFrame(StandardScaler().fit_transform(X_cluster2), columns=X_cluster2.columns)

In [26]:
print(len(X_cluster1))
print(len(X_cluster2))

1185600
1435200


In [6]:
model_cluster1 = XGBRegressor(n_estimators=500, max_depth=18, gamma=3.430739184133814, min_child_weight = 8,reg_alpha=180, reg_lambda=0.7436396623675846, random_state=123)
t1_start = perf_counter()
model_cluster1.fit(X_cluster1[:500000],y_cluster1[:500000])
t1_stop = perf_counter()
print(-(t1_start-t1_stop))

model_cluster2 = XGBRegressor(n_estimators=500, max_depth=18, gamma=3.430739184133814, min_child_weight = 8,reg_alpha=180, reg_lambda=0.7436396623675846, random_state=123)
t1_start = perf_counter()
model_cluster2.fit(X_cluster2[:500000],y_cluster2[:500000])
t1_stop = perf_counter()
print(-(t1_start-t1_stop))

[06:10:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
10995.308667290003
[09:14:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
10964.055858565


In [7]:
file_name_1 = "xgb_reg_culster1.pkl"
file_name_2 = "xgb_reg_culster2.pkl"

# save
pickle.dump(model_cluster1, open(file_name_1, "wb"))
pickle.dump(model_cluster2, open(file_name_2, "wb"))

#Val

In [8]:
X_cluster1_val, y_cluster1_val, X_cluster2_val, y_cluster2_val = data_cluster(x_val, y_val)
month1_val = X_cluster1_val['Id']
month2_val = X_cluster2_val['Id']

X_cluster1_val.drop(["month","Id"], axis=1, inplace=True)
X_cluster2_val.drop(["month","Id"], axis=1, inplace=True)

In [9]:
def m_mape(y_true,y_predict):
    n = len(y_true)
    At = np.array(y_true) + 1
    Ft = np.array(y_predict) + 1

    res = ((100/n)*(np.sum(np.abs((Ft-At)/At))))
    return res

In [10]:
X_cluster1_val = X_cluster1_val[X_cluster1.columns]
X_cluster2_val = X_cluster2_val[X_cluster2.columns]

X_cluster1_val = pd.DataFrame(StandardScaler().fit_transform(X_cluster1_val), columns=X_cluster1_val.columns)
X_cluster2_val = pd.DataFrame(StandardScaler().fit_transform(X_cluster2_val), columns=X_cluster2_val.columns)
y_pred_cluster1_val = model_cluster1.predict(X_cluster1_val)
y_pred_cluster2_val = model_cluster2.predict(X_cluster2_val) 



print("My MAPE cluster 1 =", m_mape(y_cluster1_val,y_pred_cluster1_val))
print("My MAPE cluster 2 =", m_mape(y_cluster2_val,y_pred_cluster2_val))



My MAPE cluster 1 = 11.41637549837677
My MAPE cluster 1 = 9.666196101058205


In [11]:
display(y_cluster2_val)

2880       0.0
2881       0.0
2882       0.0
2883       0.0
2884       0.0
          ... 
2845531    0.0
2845532    0.0
2845533    0.0
2845534    0.0
2845535    0.0
Name: ground_truth, Length: 1435200, dtype: float64

In [16]:
total = np.append(y_pred_cluster1_val,y_pred_cluster2_val)
total_true = np.append(y_cluster1_val,y_pred_cluster2_val)
print("My MAPE cluster =", m_mape(total,total_true))
# y_cluster1_val_pd=  pd.DataFrame(y_cluster1_val, columns=['Prediction']).reset_index()
# y_cluster2_val_pd=  pd.DataFrame(y_cluster2_val, columns=['Prediction']).reset_index()
# total_true = y_cluster1_val_pd["Prediction"].append(y_pred_cluster2_val_pd['Prediction'], ignore_index = True)


My MAPE cluster = 5.539531168489925


# Test


In [17]:
test_set = pd.read_csv("/gdrive/MyDrive/MLDM Project/data/X_all_test_final.zip")

In [7]:
file_name_1 = "xgb_reg_culster1.pkl"
file_name_2 = "xgb_reg_culster2.pkl"
model_cluster1 = pickle.load(open(file_name_1, "rb"))
model_cluster2 = pickle.load(open(file_name_2, "rb"))

[13:01:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:01:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
def data_cluster_test(X):
  # cluster_1_index = X.loc[(df['A'] > 10) & (df['B'] < 15)]
  cluster_1_index = X[(X["month"]==1)].index.values
  np.append(cluster_1_index,X[(X["month"]==2)].index)
  cluster_1_index = np.append(cluster_1_index,X[(X["month"]==3)].index)
  cluster_1_index= np.append(cluster_1_index,X[(X["month"]==4)].index)
  cluster_1_index= np.append(cluster_1_index,X[(X["month"]==11)].index)
  cluster_1_index= np.append(cluster_1_index,X[(X["month"]==12)].index)


  cluster_2_index = X[(X["month"]==5)].index.values
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==6)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==7)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==8)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==9)].index)
  cluster_2_index = np.append(cluster_2_index,X[(X["month"]==10)].index)

  X_cluster1 = X.loc[cluster_1_index]
  X_cluster2 = X.loc[cluster_2_index]

  return X_cluster1,X_cluster2


In [19]:

X_cluster1_test, X_cluster2_test = data_cluster_test(test_set)

month1 = X_cluster1_test['Id']
month2 = X_cluster2_test['Id']

X_cluster1_test.drop(["month"], axis=1, inplace=True)
X_cluster2_test.drop(["month"], axis=1, inplace=True)


In [17]:
X_cluster1_test.columns

Index(['Id', 'current hour precipitations (kg/m^2)',
       'next hour precipitation (kg/m^2)', 'station_id', 'altitude (m)',
       'latitude', 'longitude', 'latitude_idx', 'longitude_idx', 'month_cos',
       ...
       '3D_Arpege_isobar_600hpa_VerticalVelocity (Pa/s)',
       '3D_Arpege_isobar_600hpa_Geopotential (m^2/s^2)',
       '3D_Arpege_isobar_500hpa_Temperature (K)',
       '3D_Arpege_isobar_500hpa_RelavtiveHumidity (%)',
       '3D_Arpege_isobar_500hpa_WindSpeed (m/s)',
       '3D_Arpege_isobar_500hpa_WindDirection (deg)',
       '3D_Arpege_isobar_500hpa_WindComponentU (m/s)',
       '3D_Arpege_isobar_500hpa_WindComponentV (m/s)',
       '3D_Arpege_isobar_500hpa_VerticalVelocity (Pa/s)',
       '3D_Arpege_isobar_500hpa_Geopotential (m^2/s^2)'],
      dtype='object', length=107)

In [20]:
X_cluster1_test = test_set[X_cluster1.columns]
X_cluster2_test = test_set[X_cluster2.columns]

X_cluster1_test = pd.DataFrame(StandardScaler().fit_transform(X_cluster1_test), columns=X_cluster1_test.columns)
X_cluster2_test = pd.DataFrame(StandardScaler().fit_transform(X_cluster2_test), columns=X_cluster2_test.columns)
y_pred_cluster1_test = model_cluster1.predict(X_cluster1_test)
y_pred_cluster2_test = model_cluster2.predict(X_cluster2_test) 



In [ ]:
month1_pd = pd.DataFrame(month1, columns=['Id']).reset_index()
month2_pd = pd.DataFrame(month2, columns=['Id']).reset_index()

display(type(month1))
display(month1_pd)

y_pred_cluster1_test_pd =  pd.DataFrame(y_pred_cluster1_test, columns=['Prediction']).reset_index()
y_pred_cluster2_test_pd =  pd.DataFrame(y_pred_cluster2_test, columns=['Prediction']).reset_index()

y_set_1 = pd.concat([month1_pd, y_pred_cluster1_test_pd], axis=1)
print(len(y_set_1))
y_set_1.drop(["index"], axis=1, inplace=True)
y_set_2 = pd.concat([month2_pd, y_pred_cluster2_test_pd], axis=1)
y_set_2.drop(["index"], axis=1, inplace=True)
print(len(y_set_2))
pred_merged = y_set_1.append(y_set_2, ignore_index = True)
# print(len(y_set_1))
df = pred_merged.groupby("Id").agg({"Prediction": pd.Series.sum})

# display(test)
baseline = pd.read_csv("/gdrive/MyDrive/Baseline_observation_test.csv")

print("Remove Ids not in Baseline.")
submission = baseline.drop("Prediction",axis=1).merge(df, how="left", on="Id")

print(f"\nSum of NaNs :\n\n{submission.isna().sum()}\n\n")
if submission["Prediction"].isna().sum() > 0:
  print("fill nans with average.")
  submission["Prediction"].fillna(submission["Prediction"].mean(), inplace=True)

if len(submission) != 85140:
  print("Warning : len(df) != len(Baseline) i.e. {} != {}".format(len(submission), 183498))

display(submission)
print(submission.describe())
submission.to_csv('/gdrive/MyDrive/MLDM-Prediction/3Cluster_BIGfinal_xgboost_data_normalisee_Bigdata.csv', index=False)

In [25]:
display(submission)
print(submission['Prediction'].describe())

,Id,Prediction
0,14066001_149,3.686337
1,14126001_149,26.523808
2,14137001_149,7.938559
3,14216001_149,18.288322
4,14296001_149,2.363312
...,...,...
85135,86137003_293,3.248469
85136,86165005_293,2.456892
85137,86273001_293,0.021253
85138,91200002_293,-0.392913


count    85140.000000
mean         2.868358
std          2.300141
min         -0.941082
25%          1.398938
50%          2.654765
75%          3.788360
max         45.415848
Name: Prediction, dtype: float64
